<a href="https://www.kaggle.com/code/vinod123kumar/twitter-sentimental-analysis-project-ipynb?scriptVersionId=142770267" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
#Import the Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

# Data Preprocessing Stage

In [ ]:
#Load the dataset
data=pd.read_csv('/kaggle/input/hatred-on-twitter-during-metoo-movement/MeTooHate.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
#Checking the null values in the data
data.isna().sum()/len(data)*100

# Explpore Data Analysis with data

In [ ]:
'''
Create a function to checking the null values in the data with percentage
in order wise.
'''
def checking_null_values(df):
    #we creat a for loop that will return which columns have null columns
    na_columns=[col for col in df.columns if df[col].isna().sum()>0]
    # We find the null values in the null columns this values or arranged in ascending format
    na_miss=df[na_columns].isna().sum().sort_values(ascending=False)
    #and this code will be multiplication with 100 covert in to percentage
    ratio=(df[na_columns].isna().sum()/df.shape[0]*100).sort_values(ascending=False)
    # then we finally concat the miss_values and percentge with np.round 2 values and finally we create a data frame
    missing_df=pd.concat([na_miss,np.round(ratio,2)],axis=1,keys=['Null values','Percentage'])
    missing_df=pd.DataFrame(missing_df)
    return missing_df.style.background_gradient(cmap='winter_r')
checking_null_values(data)

In [ ]:
data.columns

In [ ]:
#See the numerical columns in the data
numerical=[col for col in data.columns if data[col].dtype!='object']
numerical

In [ ]:
'''
Two find the Percentage of the label data and visulize with pie chart.
in the pie chart 88% of the data is not horm and and reaming 11% is harmfull tweets.
'''
data['category'].value_counts()\
.sort_values(ascending=False)\
.plot(kind='pie',explode=[0.2,0.05],
    labels=['not-harm','harm'],
    colors=['yellow','gray'],
    autopct='%1.2f%%',
    shadow=True,title="Percentage of the harm and not-harm twittes",figsize=(16,5))
plt.show()

### Observations:
###### 1) From the above chart we visualize the label data
###### 2)We abserve 88% data is not harmful and remaing 12% is harmfull data

In [ ]:
'''To visualize the top 50 countrys most twitted
'''
data['location'].value_counts()[:50]\
.sort_values(ascending=False)\
.plot(kind='bar',title="Which location most twitted",figsize=(17,6),color=['red','green','blue','pink','orange'])
plt.xlabel("Location")
plt.ylabel("Count of location")
plt.show()

In [ ]:
data.groupby(['status_id'])['followers_count'].sum()[:50]\
.sort_values(ascending=False)\
.plot(kind='bar',figsize=(17,5),title="To visualize the follwer of the each member",color='r')
plt.xlabel("status_id")
plt.ylabel("Count of values")
plt.show()

In [ ]:
# To visualize the followers_count in the data
data['followers_count'].value_counts().sort_values(ascending=False)[:50].plot(kind='bar',figsize=(18,6),title='To visualize followers_count',color=['red','green'])
plt.xlabel('followers_count')
plt.ylabel('Count of followers')
plt.show()

In [ ]:
data.head()

In [ ]:
data['created_at']=pd.to_datetime(data['created_at'])
data['year']=data['created_at'].dt.year
data['month']=data['created_at'].dt.month
data['year'].value_counts().plot(kind='pie',explode=[0.2,0.05],
    labels=['2018','2019'],
    colors=['red','gray'],
    autopct='%1.2f%%',
    shadow=True,title="Percentage of the twitter in the year wise",figsize=(16,5))
plt.show()

## Observations:
###### 1)From the above chart 83% of the tweets tweeted in the year 2018 and 16 % twittes twitted in 2019

In [ ]:
follwer_max=data['followers_count'].max()
follwer_min=data['followers_count'].min()
friend_max=data['friends_count'].max()
friend_min=data['friends_count'].min()
print(f'the highest followers in the data is {follwer_max} ')
print(f'the highest followers in the data is {follwer_min}')
print(f'the highest friends in the data is {friend_max}')
print(f'the highest friends in the data is {friend_min}')

# Natural Language Process Techinques

In [ ]:
import string
string.punctuation

In [ ]:
#Remove the Punctuations from the data with sample question
harm=data[data['category']==1]
q1=harm['text'].values[0]
q1

In [ ]:
#We remove extra keywords in the data with sample question
test_punc_remove=[char for char in q1 if char not in string.punctuation]
test_punc_removed=''.join(test_punc_remove)
test_punc_removed

In [ ]:
test_punc_removes=[]
for char in q1:
    if char not in string.punctuation:
        test_punc_removes.append(char)
punc_remove=''.join(test_punc_removes)
punc_remove

In [ ]:
def punction_remove(text):
    if isinstance(text, float):
        return text
    else:
        test_punc_remove=[char for char in text if char not in string.punctuation]
        test_punc_removed=''.join(test_punc_remove)
        return test_punc_removed
data['text']=data['text'].apply(punction_remove)

In [ ]:
data['text']

In [ ]:
#Remove the stopwords from the data
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
stopwords=','.join(stopwords.words('english'))
stopwords

# Tokenization with nltk libaries

In [ ]:
tokenize=word_tokenize(punc_remove)
tokenize

### Stopwords Removel Process

In [ ]:
def stopwords_english(text):
    return [word for word in text if word.lower() not in stopwords]
stopwords_remove=stopwords_english(tokenize)
stopwords_remove

In [ ]:
stemmer=SnowballStemmer(language='english')
stemming_process=[stemmer.stem(word) for word in stopwords_remove]
stemming_process

In [ ]:
def stemming(text):
    stemming=[stemmer.stem(word) for word in stopwords_remove]
    stemming=''.join(stemming)
    return stemming
# lemmatizer = WordNetLemmatizer()
# q1_lem=lemmatizer.lemmatize(stemming_process)
# q1_lem

In [ ]:
data.head()

In [ ]:
def text_preprocessing(text):
    #Word_tokenize
    words=word_tokenize(text)
    #Stop words removal
    words=[word for word in words if word not in stopwords]
    #Stemming process
    words=[stemmer.stem(word) for word in words]
    #Lemmatization
#     words=[lemmatizer.lemmatize(word) for word in words]
    # Convert all words to lowercase
    words = [word.lower() for word in words]
    words=' '.join(words)
    return words

data['text'] = data['text'].apply(text_preprocessing)
